In [ ]:
from brainstates import BrainStates
import numpy as np
from scipy.signal.windows import hamming
import scipy.io as sio
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm

In [ ]:
MAIN_DIR = "/home/usuario/disco1/proyectos/2023-Coma3D"
WINDOW_SIZE = 70
STRIDE = 8
N_CLUSTERS = 3
REPLICATES = 1000
GROUP = "all"

subs = glob(f"{MAIN_DIR}/Preprocess/parcellation/DMN_all/*.mat")
subs = sorted(subs)
n_time = 500
n_rois = 34
data = np.zeros((n_time, n_rois, len(subs)))
for i, sub in enumerate(subs):
    print(sub)
    data[:,:,i] = sio.loadmat(sub)["func_roi"].T

data.shape

In [ ]:
if GROUP == "patients":
    data_conditions = {"patients": data[:,:,0:25]}
    path = f"{MAIN_DIR}/scripts/brainstates/pipeline/outputs/patients/{N_CLUSTERS}/{WINDOW_SIZE}_{STRIDE}"
else:
    data_conditions = {"patients": data[:,:,0:25], "controls": data[:,:,25:]}
    path = f"{MAIN_DIR}/scripts/brainstates/pipeline/outputs/all/{N_CLUSTERS}/{WINDOW_SIZE}_{STRIDE}"


b = BrainStates(
    from_dict=data_conditions,
    output_path=path,
    export_vars=True,
    verbose=True
)

In [ ]:
b.run(
    window_size=WINDOW_SIZE,
    stride=STRIDE,
    tapering_function=None,
    subsampling=1,
    n_clusters=N_CLUSTERS,
    n_init=REPLICATES
)


probs_patients = b.get_probs()["patients"]
if GROUP != "patients":
    probs_controls = b.get_probs()["controls"]